In [84]:
from datetime import datetime
import yfinance as yf
import time

In [85]:
class MyTradingStrategy:
    def __init__(self, name):
        self.__name = name

    def generateSignal(self , price_data):
        print("this method is intended to be over ridden")
        return "Hold"
    @property
    def name(self):
        return self.__name
    

In [86]:
obj1 = MyTradingStrategy("awesome")
obj1.name


'awesome'

In [87]:
class NewsSMATradingStrategy(MyTradingStrategy):
     def __init__(self, swindow , lwindow):
         self.__swindow = swindow
         self.__lwindow = lwindow
         super().__init__("MYSMATRADING")

     def generateSignal(self,price_data):
         if len(price_data[-self.__lwindow:]) < self.__lwindow:
            return "hold"
         short_avg = sum(price_data[-self.__swindow:])/self.__swindow 
         long_avg = sum(price_data[-self.__lwindow:])/self.__lwindow 

         if short_avg > long_avg:
            return "buy"
           
         elif short_avg < long_avg:
            return "sell"
         else:
            return "hold"

     @property
     def swindow(self):
        return self.__swindow
        
     @property
     def lwindow(self):
        return self.__lwindow

In [88]:
class TradeManage:
    def __init__(self, amount, strategyName, signal):
        self.__strategyName = strategyName
        self.__signal = signal
        self.__amount = amount
        self.__timestamp = datetime.now()  

    def execute(self):
        print(f"Execute {self.__signal} with the strategy name {self.__strategyName} and amount of {self.__amount} at time {self.__timestamp} ")

    @property
    def amount(self):
        return self.__amount
        
    @property
    def signal(self):
        return self.__signal
    @property
    def strategyName(self):
        return self.__strategyName
    @property
    def timestamp(self):
        return self.__timestamp

In [89]:
obj1 = NewsSMATradingStrategy(3 , 5)
strategy_Name = obj1.name
signal = obj1.generateSignal([1,2,3,4,5,6,7])
NewTrading = TradeManage(10000, strategy_Name, signal)
NewTrading.execute()
print(NewTrading.signal)
print(NewTrading.strategyName)
print(NewTrading.timestamp)
print(NewTrading.amount)

Execute buy with the strategy name MYSMATRADING and amount of 10000 at time 2024-07-17 19:52:52.095314 
buy
MYSMATRADING
2024-07-17 19:52:52.095314
10000


In [90]:

#Mock trading  API
trade = TradeManage(10000, strategy_Name, signal)

class MockTradingAPI:
    def __init__(self, balance):
        self.__balance = balance

    def placeOrder(self, trade, price):
        if trade.signal == "Buy" and self.balance >= trade.amount * price:
            self.__balance -= trade.amount*price 
            print(f"placed a buy trade at the price of {price} , remaining balance is {self.__balance}")
        
        elif trade.signal == "Sell":
            self.__balance += trade.amount*price
            print(f"placed a sell trade at the price of {price} , remaining balance is {self.__balance}")
        
        else:
            print("insufficient balance or invalid signal")
    @property
    def balance(self):
        return self.__balance        

In [117]:
class TradingSystem:
    def __init__(self, api, strategy, symbol):
        self.__api = api
        self.__strategy = strategy
        self.__symbol = symbol
        self.__price_data = []

    def fetchPriceData(self):
        data = yf.download(self.__symbol, period = '1d' ,  interval= '1m')
        if not data.empty:
            price = data['Close'].iloc[-1]
            self.__price_data.append(price)
            if len(self.__price_data)> self.__strategy.lwindow:
                self.__strategy.lwindow.pop(0)
                print(f"fetched new price data : {price}")
            else:
                print("no data fetched")

    def run(self):
        self.fetchPriceData()
        signal = self.__strategy.generateSignal(self.__price_data)
        print(f"generated signal is : {signal}")
        if signal in ["sell" , "buy"]:
            trade = TradeManage(1, self.__strategy, signal)
            trade.execute()
            self.api.placeOrder(trade , self.__price_data[-1])

    @property
    def api(self):
        return self.__api 
    @property
    def strategy(self):
        return self.__strategy 
    @property
    def symbol(self):
        return self.__symbol 
    @property
    def balance(self):
        return self.__price_data 
    
    

In [118]:
symbol = 'AAPL'
api = MockTradingAPI(balance = 10000)
strategy = NewsSMATradingStrategy(3,5)
system = TradingSystem(api,strategy,'AAPL')

for _ in range(3):
    system.run()
    print(f"remaining  balance :{api.balance}")
    time.sleep(2)

[*********************100%%**********************]  1 of 1 completed


no data fetched
generated signal is : hold
remaining  balance :10000


[*********************100%%**********************]  1 of 1 completed


no data fetched
generated signal is : hold
remaining  balance :10000


[*********************100%%**********************]  1 of 1 completed


no data fetched
generated signal is : hold
remaining  balance :10000


In [116]:
 data = yf.download('AAPL', period = '1d' ,  interval= '1m')
print(data)

[*********************100%%**********************]  1 of 1 completed

                                 Open        High         Low       Close  \
Datetime                                                                    
2024-07-17 09:30:00-04:00  230.130005  230.184998  229.339996  230.130005   
2024-07-17 09:31:00-04:00  230.149994  230.799896  229.570007  230.779999   
2024-07-17 09:32:00-04:00  230.794998  231.050003  230.729996  230.994995   
2024-07-17 09:33:00-04:00  231.000000  231.169998  230.500000  231.070007   
2024-07-17 09:34:00-04:00  231.070007  231.459900  230.899994  230.914993   
...                               ...         ...         ...         ...   
2024-07-17 10:31:00-04:00  227.649994  227.919907  227.550003  227.880005   
2024-07-17 10:32:00-04:00  227.910004  228.139999  227.729996  227.906403   
2024-07-17 10:33:00-04:00  227.899994  228.070007  227.830002  228.039993   
2024-07-17 10:34:00-04:00  228.039993  228.300003  228.020004  228.060104   
2024-07-17 10:35:00-04:00  228.360001  228.360001  228.360001  228.360001   